#Import packages

In [1]:
from xml.dom import minidom
from sys import stdin
from urllib import request
from subprocess import call
import numpy as np
import pyart


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



#initialize user-defined variables

In [2]:
date = "2020/01/01"
site = "KCRP"
bucketURL = "http://noaa-nexrad-level2.s3.amazonaws.com"
dirListURL = bucketURL+ "/?prefix=" + date + "/" + site

#list available files

In [3]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

print ("listing files from %s" % dirListURL)

#xmldoc = minidom.parse(stdin)
xmldoc = minidom.parse(request.urlopen(dirListURL))
itemlist = xmldoc.getElementsByTagName('Key')
print (len(itemlist) , "keys found...")

for x in itemlist:
    file = getText(x.childNodes)
    #print ("Found %s " % file)

listing files from http://noaa-nexrad-level2.s3.amazonaws.com/?prefix=2020/01/01/KCRP
257 keys found...


#read NEXRAD radar file -- having issue reading from http, temporarily reading from local directory

In [4]:
#https://s3.amazonaws.com/noaa-nexrad-level2/index.html#2020/01/01/KCRP/
#https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/KCRP20200101_000431_V06

#radar = pyart.io.read('https://noaa-nexrad-level2.s3.amazonaws.com/2020/01/01/KCRP/' + 'KCRP20200101_000431_V06')
radar = pyart.io.read('KCRP20200101_000431_V06')
radar.info()

altitude:
	data: <ndarray of type: float64 and shape: (1,)>
	long_name: Altitude
	standard_name: Altitude
	units: meters
	positive: up
altitude_agl: None
antenna_transition: None
azimuth:
	data: <ndarray of type: float64 and shape: (6480,)>
	units: degrees
	standard_name: beam_azimuth_angle
	long_name: azimuth_angle_from_true_north
	axis: radial_azimuth_coordinate
	comment: Azimuth of antenna relative to true north
elevation:
	data: <ndarray of type: float32 and shape: (6480,)>
	units: degrees
	standard_name: beam_elevation_angle
	long_name: elevation_angle_from_horizontal_plane
	axis: radial_elevation_coordinate
	comment: Elevation of antenna relative to the horizontal plane
fields:
	spectrum_width:
		data: <ndarray of type: float32 and shape: (6480, 1832)>
		units: meters_per_second
		standard_name: doppler_spectrum_width
		long_name: Spectrum Width
		valid_max: 63.0
		valid_min: -63.5
		coordinates: elevation azimuth range
		_FillValue: -9999.0
	reflectivity:
		data: <ndarray of typ

#confirm station location

In [5]:
altitude0 = radar.altitude['data'][0]
latitude0 = radar.latitude['data'][0]
longitude0 = radar.longitude['data'][0]
print(altitude0, latitude0, longitude0)

43.0 27.78401756286621 -97.5112533569336


In [37]:
print(radar.nrays)
print(radar.ngates)

6480
1832


#Ray attributes: azimuth, elevation, time
#Gate attributes: range
#Ray x Gate attributes: lat, lon, alt

In [36]:
gate_lat = radar.gate_latitude['data']
gate_lon = radar.gate_longitude['data']
gate_alt = radar.gate_altitude['data']
print(len(radar.gate_latitude['data']))
print(len(radar.gate_latitude['data'][0]))

import itertools
merged_lat = list(itertools.chain.from_iterable(gate_lat))
merged_lon = list(itertools.chain.from_iterable(gate_lon))
merged_alt = list(itertools.chain.from_iterable(gate_alt))

6480
1832


In [26]:
refl = radar.fields['reflectivity']['data']

In [27]:
merged_refl = list(itertools.chain.from_iterable(refl))

In [28]:
len(merged_refl)

11871360

In [30]:
radar.time['data']

array([1.83000e-01, 2.79000e-01, 3.79000e-01, ..., 4.07807e+02,
       4.07860e+02, 4.07914e+02])

In [16]:
#print(len(radar.time['data']))
#sample_time = radar.time['data']
#merged_time = list(itertools.chain.from_iterable(sample_time))

TypeError: 'numpy.float64' object is not iterable

In [ ]:
import pandas as pd
#gate_lla = zip(gate_lat, gate_lon, gate_alt)
gate_lla = pd.DataFrame(
    {'Gate latitude': gate_lat,
     'Gate longitude': gate_lon,
     'Gate altitude': gate_alt
    })

In [ ]:
#sample_lat_dict = {'data': sample_lat, 'units': 'test', 'long_name': 'sample_latitude',
             '_FillValue': sample_lat.fill_value, 'standard_name': 'sample_latitude'}

#radar.add_field('sample_latitude', sample_lat_dict, replace_existing=True)